In [1]:
# Pandas dataframe
import pandas as pd
from pandas import DataFrame

# Scikit-Learn
import sklearn 
from sklearn import datasets, linear_model, metrics, tree
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score

# Models
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.ensemble import (BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, 
RandomForestRegressor, AdaBoostRegressor)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso,  RidgeCV
from sklearn.neighbors import KNeighborsRegressor
        
import joblib
from matplotlib import pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [2]:
df_size = pd.read_csv("dataset_scaled_size.csv")
df_size

,Unnamed: 0,atomic_number,radius_pm,oxid_state,relativistic,metal_amount_mmol,num of group,spcaer,anchoring_group,chirality type,...,aromaticity1,sol1_vol_mL,protic2,polarity index 2,aromaticity2,solv2_vol_mL,time_min,temp_c,size_nm,g_factor*10^4
0,0,0.90070,-0.626518,0.980887,0.900450,-0.454943,-0.280991,-0.526910,-0.803389,0.120568,...,-0.070711,-0.124583,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,3.06,0.000000
1,1,0.90070,-0.626518,0.980887,0.900450,-0.454943,-0.280991,-0.526910,-0.803389,0.120568,...,-0.070711,-0.104117,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,1.00,0.757989
2,2,-1.03686,-0.252283,-0.853143,-1.110555,1.601355,0.049297,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,-0.537086,-0.793018,-0.240613,-0.523310,-0.593509,-0.997007,1.00,0.583069
3,3,-1.03686,-0.252283,-0.853143,-1.110555,1.768534,-0.280991,-0.526910,-0.803389,0.120568,...,-0.070711,-0.032487,-0.537086,-0.793018,-0.240613,-0.523310,-0.593509,-0.997007,2.37,0.000000
4,4,0.90070,-0.626518,0.980887,0.900450,-0.454943,0.049297,-0.526910,-0.803389,0.120568,...,-0.070711,-0.124583,-0.537086,-0.793018,-0.240613,-0.523310,0.680663,-0.997007,4.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,196,-1.03686,-0.252283,-0.853143,-1.110555,-0.265474,-0.115847,-0.742392,-0.803389,0.120568,...,-0.070711,-0.605526,-0.537086,-0.793018,-0.240613,-0.523310,0.217328,-0.176375,NaN,2.000000
197,197,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.05,15.000000
198,198,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.30,10.000000
199,199,-1.03686,-0.252283,-0.853143,-1.110555,0.570420,-0.115847,-0.526910,-0.803389,0.120568,...,-0.070711,-0.175747,1.861899,1.679991,-0.240613,-0.004855,-0.535592,-0.176375,1.05,7.000000


In [3]:
input_col = [col for col in df_size.columns]
input_col.remove('Unnamed: 0')
input_col.remove('size_nm')
input_col.remove('g_factor*10^4')
input_col

['atomic_number',
 'radius_pm',
 'oxid_state',
 'relativistic',
 'metal_amount_mmol',
 'num of group',
 'spcaer',
 'anchoring_group',
 'chirality type',
 'hydrogen bond',
 'aromaticity',
 'ligand_amount_mmol',
 'redu_num',
 'reductant_amount_mmol',
 'protic1',
 'polarity index1',
 'aromaticity1',
 'sol1_vol_mL',
 'protic2',
 'polarity index 2',
 'aromaticity2',
 'solv2_vol_mL',
 'time_min',
 'temp_c']

In [4]:
output_col = ['size_nm']

X = df_size[input_col]

Y = df_size[output_col]

In [5]:
# Splitting dataset for training
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=45, shuffle=True)

## Bagging

In [6]:
min_mae = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 20)):
    for j in range(1, 20):
        for k in range(2, 36, 2):
            
            B_regr = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=i),
                                      n_estimators=j,
                                      random_state=k)
            
            B_regr.fit(X_train, np.ravel(Y_train))
            
            B_Y_pred = B_regr.predict(X_test)
            
            mae = mean_absolute_error(Y_test, B_Y_pred)
            
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 19/19 [02:09<00:00,  6.81s/it]

0.9388309178743961 12 3 30


## Random Forest

In [7]:
min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 26)):
    for j in range(1, 26):
        for k in range(2, 23, 2):
            RF_regr = RandomForestRegressor(max_depth=i, 
                                            n_estimators=j, 
                                            max_features=k,
                                            random_state=45
                                                )
            RF_regr.fit(X_train, np.ravel(Y_train))
            RF_Y_pred = RF_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, RF_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 25/25 [02:01<00:00,  4.88s/it]

0.9293488721423502 10 4 22


## Decision Tree

In [8]:
min_mae = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 26)):
    for j in range(1, 23):
        for k in range(2, 45, 1):
            
            DT_regr = DecisionTreeRegressor(max_depth=i,
                                max_features=j,
                                random_state=k)
            
            DT_regr.fit(X_train, Y_train)

            DT_Y_pred = DT_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, DT_Y_pred)
            
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 25/25 [01:37<00:00,  3.89s/it]

0.6515256603300081 8 15 44


## Extra Trees

In [9]:
min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 30)):
    for j in range(1, 23):
        for k in range(2, 40, 1):
            ET_regr = ExtraTreesRegressor(n_estimators=i, 
                                            max_features=j,
                                            random_state=k
                                                )
            ET_regr.fit(X_train, np.ravel(Y_train))
            ET_Y_pred = ET_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, ET_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 29/29 [06:32<00:00, 13.54s/it]

0.7817391304347826 2 5 2


## AdaBoost

In [10]:
min_mae = 99999
min_i, min_j = 0, 0
for i in tqdm(range(1, 30)):
    for j in range(1, 30):
        AB_regr = AdaBoostRegressor(n_estimators=i, loss='exponential', learning_rate=j)
        AB_regr.fit(X_train, np.ravel(Y_train))
        AB_Y_pred = AB_regr.predict(X_test)
        AB_mae = mean_absolute_error(Y_test, AB_Y_pred)
        if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
print(min_mae, min_i, min_j)

100%|██████████| 29/29 [00:15<00:00,  1.90it/s]

2.0886919040479763 1 1


## Gradient Boosting

In [11]:
'''
min_mae = 999
min_i, min_j, min_k, min_l = 0, 0, 0.0, 0
for i in tqdm(range(300, 450, 10)):
    for j in range(2, 30, 2):
        for k in np.arange(0.06, 0.22, 0.02):
            for l in range(2, 22, 2):
                GB_regr = GradientBoostingRegressor(n_estimators=i, max_depth=j, learning_rate=k, random_state=l)
                GB_regr.fit(X_train, np.ravel(Y_train))
                GB_Y_pred = GB_regr.predict(X_test)

                mae = mean_absolute_error(Y_test, GB_Y_pred)
                if (min_mae > mae):
                    min_mae = mae
                    min_i = i
                    min_j = j
                    min_k = k
                    min_l = l

print(min_mae, min_i, min_j, min_k, min_l)
'''

'\nmin_mae = 999\nmin_i, min_j, min_k, min_l = 0, 0, 0.0, 0\nfor i in tqdm(range(300, 450, 10)):\n    for j in range(2, 30, 2):\n        for k in np.arange(0.06, 0.22, 0.02):\n            for l in range(2, 22, 2):\n                GB_regr = GradientBoostingRegressor(n_estimators=i, max_depth=j, learning_rate=k, random_state=l)\n                GB_regr.fit(X_train, np.ravel(Y_train))\n                GB_Y_pred = GB_regr.predict(X_test)\n\n                mae = mean_absolute_error(Y_test, GB_Y_pred)\n                if (min_mae > mae):\n                    min_mae = mae\n                    min_i = i\n                    min_j = j\n                    min_k = k\n                    min_l = l\n\nprint(min_mae, min_i, min_j, min_k, min_l)\n'

## Others

In [12]:
REGRESSIONS = {
    "K-nn": KNeighborsRegressor(),                          
    "Ridge": RidgeCV(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=0),
                }
# mean absolute error is used to evaluate the performance of all regressions.


for name, reg in REGRESSIONS.items():     
    reg.fit(X_train, Y_train)                 
    Y_pred = pd.DataFrame(reg.predict(X_test))
    
    print(name)
    
    mae = mean_absolute_error(Y_test, Y_pred)
    
    print(' MAE for Abs is ', mae, '\n')

K-nn
 MAE for Abs is  3.615043478260869 

Ridge
 MAE for Abs is  4.585248314922484 

Lasso
 MAE for Abs is  3.580829616464104 

ElasticNet
 MAE for Abs is  4.632486966871571 



## Saving the best model

In [14]:
DT_regr = DecisionTreeRegressor(max_depth=8,
                                max_features=15,
                                random_state=44)
            
DT_regr.fit(X_train, Y_train)
DT_Y_pred = DT_regr.predict(X_test)

joblib.dump(DT_regr, "./model_aug_size_DecisionTree.joblib")

['./model_aug_size_DecisionTree.joblib']